In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy  
import pandas as pd  
import random
import os
import re


print('-'*100)
print('연습문제 8-1 : 네이버 영화 리뷰 정보 수집하는 크롤러')
print('='*100)

movie_title = input('1. 크롤링 할 영화의 제목을 입력하세요 : ')
cnt = int(input('2. 크롤링 할 리뷰건수는 몇건입니까? : '))

page_cnt = math.ceil(cnt/10)

now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

f_dir = input('3. 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder2/) : ')
if f_dir == "" : 
    f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
#저장경로 설정
os.makedirs(f_dir+s+'-'+movie_title)
os.chdir(f_dir+s+'-'+movie_title)

print('-'*100)

#파일형식
ff_name = s+'-'+movie_title+'.txt'
fc_name = s+'-'+movie_title+'.csv'
fx_name = s+'-'+movie_title+'.xlsx'

s_time = time.time()
#드라이버 호출
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('disable-gpu')
options.add_argument('lang=ko_KR')
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
driver = webdriver.Chrome(path, options=options)

driver.get('https://movie.naver.com/')
time.sleep(random.randrange(2, 5))

element1 = driver.find_element_by_class_name('ipt_tx_srch')
element1.send_keys(movie_title)
element1.send_keys('\n')
time.sleep(3)

no = 0
#페이지에 조회된 여러 영화 중에 선택하게 하기 
html0 = driver.page_source
soup0 = BeautifulSoup(html0, 'html.parser')
dt_tag_list0 = soup0.find('ul', 'search_list_1').find_all('dt')

dt_number = len(dt_tag_list0)

url_list = []
for a in dt_tag_list0 : 
    a_tag = a.find('a')
    a_text = a_tag.get_text().replace('/n', "").strip()
    print(no,":", a_text)
    no += 1
    hl = a_tag['href']
    url_list.append(hl)
    
driver.quit()

try : 
    select = int(input('%s 개 영화가 검색되었습니다. 위 목록에서 영화 번호를 하나만 선택해주세요 : ' %dt_number))
except : 
    try : 
        print('숫자 하나만 입력하셔야 합니다.')
        select = int(input('%s 개 영화가 검색되었습니다. 위 목록에서 영화 번호를 하나만 선택해주세요 : ' %dt_number))
    except : 
        raise('TypeError')
selected_url = url_list[select]

#영화 창 열기 
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('lang=ko_KR')
options.add_argument('disable-gpu')
options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')


path = '/Users/kibeomkim/Desktop/chromedriver_240/chromedriver'
driver = webdriver.Chrome(path, options=options)
driver.get('https://movie.naver.com'+selected_url)# 원하는 영화페이지까지 접속완료. 수집만 하면 된다. 
time.sleep(random.randrange(3, 5))

#댓글 내용 수집하기
driver.find_element_by_xpath('/html/body/div/div[4]/div[3]/div[1]/div[3]/ul/li[5]/a').click()
time.sleep(3)

driver.switch_to.frame('pointAfterListIframe')
time.sleep(3)
html1 = driver.page_source 
soup1 = BeautifulSoup(html1, 'html.parser')
total_number = int(soup1.find('div', 'score_total').find('em').get_text().replace(',', "").strip())

#토탈이랑 비교해서 cnt 조정
if cnt > total_number : 
    print('%s건 수집 요청하셨으나 %s 건 밖에 존재하지 않아 %s 건 만큼만 수집하겠습니다' %(cnt, total_number, total_number))
    cnt = total_number 

result_list = soup1.find('div', 'score_result').find_all('li')
time.sleep(2)

no1 = 1
f = open(ff_name, 'a', encoding='utf-8')
star_list1 = []
review_list1 = []
writer1 = []
date_list1 = []
em1 = []
bem1 = []


for c in range(1, (page_cnt+1)) : 
    for b in result_list : 
        print('\n')
        print('총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다=================' %(cnt, no1))
        f.write('총 %s 건 중 %s 번째 리뷰 데이터를 수집합니다=================' %(cnt, no1))
    
        star_score = b.find('div', 'star_score').find('em').get_text().strip()
        f.write('1.별점:'+star_score + '\n')
        star_list1.append(star_score)
        print('1.별점:'+star_score+'\n')
    
        content = b.find('div', 'score_reple').find('p').get_text().replace('\n', "").replace('\n', "").strip()
        f.write('2.리뷰내용:'+content + '\n')
        review_list1.append(content)
        print('2.리뷰내용:'+content+'\n')
    
        writer = b.find('div', 'score_reple').find('a').get_text().strip()
        f.write('3.작성자:'+writer+'\n')
        writer1.append(writer)
        print('3.작성자:'+writer+'\n')
    
        date = b.find('div', 'score_reple').find('dt').get_text().replace('\n', "").replace(writer, "").strip()
        f.write('4.작성일자:'+date+'\n')
        date_list1.append(date)
        print('4.작성일자:'+date+'\n')
    
        em = b.find('div', 'btn_area').find('a','_sympathyButton').find('strong').get_text().strip()
        f.write('5.공감:'+em+'\n')
        em1.append(em)
        print('5.공감:'+em+'\n')
    
        bem = b.find('div', 'btn_area').find('a','_notSympathyButton').find('strong').get_text().strip()
        f.write('6.비공감:'+bem+'\n')
        f.write('\n')
        bem1.append(bem)
        print('6.비공감:'+bem+'\n')
        print('%s 건 완료===============================' %no1)
        print('\n')
        time.sleep(2)
        
        no1 +=1 
        if no1 > cnt : 
            break
    c += 1
    if c > page_cnt : 
        break 
    
    try : 
        driver.find_element_by_id('pagerTagAnchor%s' %c).click()
        time.sleep(3)
    except : 
        print('더 이상 페이지가 없습니다')
f.close()
driver.quit()

movie = pd.DataFrame()
movie['별점(평점)'] = star_list1
movie['리뷰내용'] = review_list1
movie['작성자'] = writer1
movie['작성일자'] = date_list1
movie['공감횟수'] = em1
movie['비공감횟수'] = bem1

movie.to_csv(fc_name, encoding='utf-8-sig')
movie.to_excel(fx_name)

e_time = time.time()
total_time = e_time - s_time

print('='*100)
print('총 %s 건 조회했습니다' %cnt)
print('총 소요시간은 %s 초 입니다' %total_time)
print('='*100)

    



----------------------------------------------------------------------------------------------------
연습문제 8-1 : 네이버 영화 리뷰 정보 수집하는 크롤러
1. 크롤링 할 영화의 제목을 입력하세요 : 엔드게임
2. 크롤링 할 리뷰건수는 몇건입니까? : 100
3. 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder2/) : 
----------------------------------------------------------------------------------------------------
0 : 엔드게임 (ENDGAME)
1 : 어벤져스: 엔드게임 (Avengers: Endgame)
2 : 엔드 게임 (End Game)
3 : 앤드게임 (endgame)
4 : 엔드게임 (Endgame)
5 개 영화가 검색되었습니다. 위 목록에서 영화 번호를 하나만 선택해주세요 : 1


총 100 건 중 1 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:마블 3000만큼 사랑합니다

3.작성자:전은호(eunh****)

4.작성일자:2019.04.24 10:31

5.공감:23705

6.비공감:663

1 건 완료===============================




총 100 건 중 2 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:Avengers Assemble!

3.작성자:송(hiuh****)

4.작성일자:2019.04.24 09:35

5.공감:15700

6.비공감:442

2 건 완료===============================




총 100 건 중 3 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:I'm Ironman.

3.작성자:ubola(ubol****)

4.작성일자



총 100 건 중 38 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:쥐가다살렸음 ㅋㅋㅋㅋ

3.작성자:에헴(krth****)

4.작성일자:2019.04.24 15:26

5.공감:8992

6.비공감:258

38 건 완료===============================




총 100 건 중 39 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:관람객																																																												마지막 한타에서 쌌습니다

3.작성자:김대훈(eogn****)

4.작성일자:2019.04.24 10:51

5.공감:8159

6.비공감:366

39 건 완료===============================




총 100 건 중 40 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:10점 안주는 사람들은 마블에 6에서900까지밖에 애정이 없는거 ...

3.작성자:tlsg****

4.작성일자:2019.04.24 11:50

5.공감:7603

6.비공감:441

40 건 완료===============================




총 100 건 중 41 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:마블 3000만큼 사랑합니다

3.작성자:전은호(eunh****)

4.작성일자:2019.04.24 10:31

5.공감:23705

6.비공감:663

41 건 완료===============================




총 100 건 중 42 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:Avengers Assemble!

3.작성자:송(hiuh****)

4.작성일자:2019.04.24 09:35

5.공감:15700

6.비공감:442

42 건 완료=



총 100 건 중 77 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:인피니티워가 10년을 무너뜨리는 영화라면엔드게임은 10년을 돌아보는 영화입니다

3.작성자:메주0228693848(wook****)

4.작성일자:2019.04.24 12:20

5.공감:9728

6.비공감:260

77 건 완료===============================




총 100 건 중 78 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:쥐가다살렸음 ㅋㅋㅋㅋ

3.작성자:에헴(krth****)

4.작성일자:2019.04.24 15:26

5.공감:8992

6.비공감:258

78 건 완료===============================




총 100 건 중 79 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:관람객																																																												마지막 한타에서 쌌습니다

3.작성자:김대훈(eogn****)

4.작성일자:2019.04.24 10:51

5.공감:8159

6.비공감:366

79 건 완료===============================




총 100 건 중 80 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:10점 안주는 사람들은 마블에 6에서900까지밖에 애정이 없는거 ...

3.작성자:tlsg****

4.작성일자:2019.04.24 11:50

5.공감:7603

6.비공감:441

80 건 완료===============================




총 100 건 중 81 번째 리뷰 데이터를 수집합니다=================
1.별점:10

2.리뷰내용:마블 3000만큼 사랑합니다

3.작성자:전은호(eunh****)

4.작성일자:2019.04.24 10: